In [33]:
# set up the data set
from datetime import time
import pandas as pd

data = pd.read_excel(r"newInput.xlsx")
df = pd.DataFrame(data)

In [34]:
"""defining functions to converts to seconds and then  back to time object """
def convertToMin(time_obj):
    return (time_obj.hour * 60 + time_obj.minute)

def convertToTime(minutes):
    hour, minute = divmod(minutes, 60)
    return time(hour, minute)

In [35]:
# can create a additional function to call this in future development 
strToTime = lambda strTime: time(*[int(x) for x in strTime.split(':')])
# DEFINING GLOBAL VARIABLES 
sorted_set = df.sort_values('Departure Time', ascending=True).reset_index(drop=True)
start_set = sorted_set[sorted_set['Departure Place'] == 'PSL'].reset_index(drop=True)

# GLOBAL VARIABLES
# all the time constraints are mentioned in minutes
minDuty = 8 * 60 
maxDuty = 10 * 60 
# minSpreadOver = 10 * 60 
maxSpreadOver = 12 * 60 
trip = [{"selected_leg":start_set.iloc[0].to_frame().T, 
         "temp_set": start_set.drop(0).reset_index(drop=True),
         "duty": convertToMin(strToTime(start_set.iloc[0]['Running Time'])),
         }]

In [36]:
def getSpreadOver():
    global trip
    start = strToTime(trip[0]['selected_leg']['Departure Time'][0])
    end = strToTime(trip[-1]['selected_leg']['Arrival Time act'][0])
    return (convertToMin(end) - convertToMin(start))

def setDuty(leg):
    global trip 
    duty = trip[-1]['duty']
    duty += convertToMin(strToTime(leg['Running Time'][0]))
    last_arrival = convertToMin(strToTime(trip[-1]['selected_leg']['Arrival Time act'][0]))
    new_departure = convertToMin(strToTime(leg['Departure Time'][0]))
    terminal_gap = new_departure - last_arrival
    if terminal_gap < 15:
        duty += terminal_gap
    return (terminal_gap, duty)


def getCurrentTrip():
    global trip
    route = [x['selected_leg'] for x in trip]
    display(pd.concat(route).reset_index(drop=True))
    print("Steering Duty = ", convertToTime(trip[-1]['duty']))
    print("Spread Over = ", convertToTime(getSpreadOver()))



In [37]:
def pushToTrip():
    global trip
    while trip[-1]['selected_leg']['Arrival Place'][0] != 'PSL':
        last_leg = trip[-1]['selected_leg']
        temp_set = df[(df['Departure Place'] == last_leg['Arrival Place'][0]) & (
            df['Departure Time'] > last_leg['Arrival Time act'][0])]
        temp_set = temp_set.sort_values(
            'Departure Time', ascending=True).reset_index(drop=True)
        new_leg = temp_set.iloc[0].to_frame().T
        terminal_gap, duty = setDuty(new_leg)
        # print(terminal_gap, ", ",  duty)
        trip[-1]['selected_leg']['Terminal Gap'] = terminal_gap
        trip.append({"selected_leg": new_leg,
                     "temp_set": temp_set.drop(0).reset_index(drop=True), 
                     "duty": duty, }
                    )


pushToTrip()
getCurrentTrip()
# what happens if the first trip itself isn't valid 

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,5.0
5,695,10:05:00,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,13:35:00,81.5,03:30:00,NaN


Steering Duty =  09:55:00
Spread Over =  09:55:00


In [38]:
for i in range(50):
    temp = trip.pop()
    terminal_gap, duty = setDuty(temp['temp_set'].iloc[0].to_frame().T)
    trip[-1]['selected_leg']['Terminal Gap'] = terminal_gap
    if duty < maxDuty:
        temp['selected_leg'] = temp['temp_set'].iloc[0].to_frame().T
        temp['temp_set'] = (temp['temp_set'].drop(0)).reset_index(drop=True)
        temp['duty'] = duty
        trip.append(temp)
    else:
        temp = trip.pop()
        # terminal_gap, duty = setDuty(temp['temp_set'].iloc[0].to_frame().T)
        # trip[-1]['selected_leg']['Terminal Gap'] = terminal_gap
        # temp['selected_leg'] = temp['temp_set'].iloc[0].to_frame().T
        # temp['temp_set'] = (temp['temp_set'].drop(0)).reset_index(drop=True)
        # temp['duty'] = duty
        # trip.append(temp)

    pushToTrip()
    print("Trip ", i)
    getCurrentTrip()

Trip  0


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,10.0
5,665,10:10:00,KLKV,PVR-VZM-BYPASS-TVM-KANMLA,MC,12:25:00,55.0,02:15:00,5.0
6,347,12:30:00,MC,NH,KLKV,14:10:00,40.0,01:40:00,5.0
7,91,14:15:00,KLKV,NH,PSL,14:25:00,3.5,00:10:00,NaN


Steering Duty =  10:45:00
Spread Over =  10:45:00
Trip  1


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,10.0
5,665,10:10:00,KLKV,PVR-VZM-BYPASS-TVM-KANMLA,MC,12:25:00,55.0,02:15:00,5.0
6,347,12:30:00,MC,NH,KLKV,14:10:00,40.0,01:40:00,5.0
7,91,14:15:00,KLKV,NH,PSL,14:25:00,3.5,00:10:00,NaN


Steering Duty =  10:45:00
Spread Over =  10:45:00
Trip  2


...

Steering Duty =  10:45:00
Spread Over =  10:45:00
Trip  48


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,10.0
5,665,10:10:00,KLKV,PVR-VZM-BYPASS-TVM-KANMLA,MC,12:25:00,55.0,02:15:00,5.0
6,347,12:30:00,MC,NH,KLKV,14:10:00,40.0,01:40:00,5.0
7,91,14:15:00,KLKV,NH,PSL,14:25:00,3.5,00:10:00,NaN


Steering Duty =  10:45:00
Spread Over =  10:45:00
Trip  49


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,10.0
5,665,10:10:00,KLKV,PVR-VZM-BYPASS-TVM-KANMLA,MC,12:25:00,55.0,02:15:00,5.0
6,347,12:30:00,MC,NH,KLKV,14:10:00,40.0,01:40:00,5.0
7,91,14:15:00,KLKV,NH,PSL,14:25:00,3.5,00:10:00,NaN


Steering Duty =  10:45:00
Spread Over =  10:45:00


In [39]:

""" 
Sl No.
Departure Time
Departure Place
Route of Operation
Arrival Place
Arrival Time act
KM
Running Time 
"""

' \nSl No.\nDeparture Time\nDeparture Place\nRoute of Operation\nArrival Place\nArrival Time act\nKM\nRunning Time \n'